## Vectorbt - Dual Bollinger Bands on single asset

Reference 1: https://greyhoundanalytics.com/blog/create-a-custom-indicator-in-vectorbt/ 

Reference 2: http://qubitquants.pro/strategydev/index.html

In [65]:
import vectorbt as vbt
import numpy

In [104]:
cols = ['Open', 'High', 'Low', 'Close', 'Volume']
ohlcv = vbt.YFData.download("BTC-USD", missing_index="drop",
                            start="07-08-2023",
                            interval="5m").get(cols)
print(ohlcv)

                                   Open          High           Low  \
Datetime                                                              
2023-07-07 16:00:00+00:00  30410.642578  30412.316406  30403.056641   
2023-07-07 16:05:00+00:00  30391.021484  30396.808594  30385.951172   
2023-07-07 16:10:00+00:00  30395.443359  30434.644531  30395.443359   
2023-07-07 16:15:00+00:00  30414.837891  30414.837891  30387.546875   
2023-07-07 16:20:00+00:00  30387.925781  30387.925781  30379.187500   
...                                 ...           ...           ...   
2023-08-08 13:25:00+00:00  29539.755859  29547.119141  29474.988281   
2023-08-08 13:30:00+00:00  29473.226562  29478.996094  29435.845703   
2023-08-08 13:35:00+00:00  29429.650391  29429.650391  29367.394531   
2023-08-08 13:40:00+00:00  29388.015625  29391.062500  29366.275391   
2023-08-08 13:45:00+00:00  29369.138672  29373.371094  29369.138672   

                                  Close     Volume  
Datetime               

In [113]:
ohlcv["Close"].resample("1h").asfreq()

Datetime
2023-07-07 16:00:00+00:00    30403.056641
2023-07-07 17:00:00+00:00    30356.871094
2023-07-07 18:00:00+00:00    30247.949219
2023-07-07 19:00:00+00:00    30174.451172
2023-07-07 20:00:00+00:00    30228.275391
                                 ...     
2023-08-08 09:00:00+00:00    29144.730469
2023-08-08 10:00:00+00:00    29164.953125
2023-08-08 11:00:00+00:00    29310.738281
2023-08-08 12:00:00+00:00    29343.025391
2023-08-08 13:00:00+00:00    29611.146484
Freq: H, Name: Close, Length: 766, dtype: float64

In [116]:
ohlcv["High"].resample("4h").last()

Datetime
2023-07-07 16:00:00+00:00    30225.230469
2023-07-07 20:00:00+00:00    30338.564453
2023-07-08 00:00:00+00:00    30287.886719
2023-07-08 04:00:00+00:00    30201.736328
2023-07-08 08:00:00+00:00    30208.083984
                                 ...     
2023-08-07 20:00:00+00:00    29183.974609
2023-08-08 00:00:00+00:00    29179.650391
2023-08-08 04:00:00+00:00    29160.099609
2023-08-08 08:00:00+00:00    29397.947266
2023-08-08 12:00:00+00:00    29373.371094
Freq: 4H, Name: High, Length: 192, dtype: float64

In [119]:
h4_BBANDS = vbt.BBANDS.run(ohlcv["Close"], window=240)
m15_BBANDS = vbt.BBANDS.run(ohlcv["Close"], window=15)
m15_RSI = vbt.RSI.run(ohlcv["Close"], window=15) # 15 * 1min = 15 min RSI

entry
1. H4 "Low" price goes below its lower Bollinger band
2. 15m RSI goes below its lower Bollinger band

exit
1. H4 "High" price breaks its upper Bollinger band
2. 15m RSI breaks above its upper Bollinger band

In [122]:
## The two variables `bb_upper_fract` and `bb_lower_fract` are simply some adjustment 
# parameters for the RSI bollinger bands and they are explained at the end of this article.

entries = h4_BBANDS.lower_above(ohlcv["Low"])\
                & m15_RSI.rsi_below(m15_BBANDS.upper)
exits = h4_BBANDS.upper_below(ohlcv["High"])\
                    & m15_RSI.rsi_above(m15_BBANDS.upper)

In [127]:
ohlcv["Low"]

Datetime
2023-07-07 16:00:00+00:00    30403.056641
2023-07-07 16:05:00+00:00    30385.951172
2023-07-07 16:10:00+00:00    30395.443359
2023-07-07 16:15:00+00:00    30387.546875
2023-07-07 16:20:00+00:00    30379.187500
                                 ...     
2023-08-08 13:25:00+00:00    29474.988281
2023-08-08 13:30:00+00:00    29435.845703
2023-08-08 13:35:00+00:00    29367.394531
2023-08-08 13:40:00+00:00    29366.275391
2023-08-08 13:45:00+00:00    29369.138672
Freq: 5T, Name: Low, Length: 9190, dtype: float64

In [126]:
m15_BBANDS.middle.shape

(9190,)

In [123]:
exits.sum()

0

In [125]:
entries.sum()

685